## Gradient-Boosting anything (alert: high performance)

<a target="_blank" href="https://colab.research.google.com/github/Techtonique/mlsauce/blob/master/mlsauce/demo/thierrymoudiki-2024-10-06-genericboosting.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

We've always been told that decision trees are _best_ for [Gradient Boosting](https://en.wikipedia.org/wiki/Gradient_boosting) Machine Learning. I've always wanted to see for myself. [AdaBoostClassifier](https://techtonique.github.io/nnetsauce/nnetsauce.html#AdaBoostClassifier) is working well, but is relatively _slow_ (by my own standards). A few days ago, I noticed that my Cython implementation of [LSBoost](https://www.researchgate.net/publication/346059361_LSBoost_gradient_boosted_penalized_nonlinear_least_squares) in [mlsauce]() was already quite _generic_ (never noticed before), and I decided to adapt it to any machine learning model with `fit` and `predict` methods. It's worth mentioning that only regression algorithms are accepted, and classification is [regression-based](https://www.researchgate.net/publication/377227280_Regression-based_machine_learning_classifiers). The results are promising indeed; I'll let you see for yourself below. All the algorithms, including `xgboost` and `RandomForest`, are used with their default hyperparameters. Which means, there's still a room for improvement.



In [1]:
!pip install git+https://github.com/Techtonique/mlsauce.git --verbose --upgrade --no-cache-dir

Using pip 24.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
  Cloning https://github.com/Techtonique/mlsauce.git to /tmp/pip-req-build-2q8062xh
  Running command git version
  git version 2.34.1
  Running command git clone --filter=blob:none https://github.com/Techtonique/mlsauce.git /tmp/pip-req-build-2q8062xh
  Cloning into '/tmp/pip-req-build-2q8062xh'...
  Updating files:   1% (2/136)
  Updating files:   2% (3/136)
  Updating files:   3% (5/136)
  Updating files:   4% (6/136)
  Updating files:   5% (7/136)
  Updating files:   6% (9/136)
  Updating files:   7% (10/136)
  Updating files:   8% (11/136)
  Updating files:   9% (13/136)
  Updating files:  10% (14/136)
  Updating files:  11% (15/136)
  Updating files:  12% (17/136)
  Updating files:  13% (18/136)
  Updating files:  14% (20/136)
  Updating files:  15% (21/136)
  Updating files:  16% (22/136)
  Updating files:  17% (24/136)
  Updating files:  18% (25/136)
  Updating files:  19% (26/136)
  Updating files:

In [ ]:
import os
import pandas as pd
import mlsauce as ms
from sklearn.datasets import load_breast_cancer, load_iris, load_wine, load_digits
from sklearn.model_selection import train_test_split
from time import time

load_models = [load_breast_cancer, load_wine, load_iris]

for model in load_models:

    data = model()
    X = data.data
    y= data.target
    X = pd.DataFrame(X, columns=data.feature_names)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state = 13)

    clf = ms.LazyBoostingClassifier(verbose=0, ignore_warnings=True,
                                    custom_metric=None, preprocess=False)

    start = time()
    models, predictions = clf.fit(X_train, X_test, y_train, y_test)
    print(f"\nElapsed: {time() - start} seconds\n")

    display(models)


2it [00:01,  1.52it/s]
100%|██████████| 30/30 [00:21<00:00,  1.38it/s]


Elapsed: 23.019137859344482 seconds



Accuracy  Balanced Accuracy  \
Model                                                                     
GenericBooster(LinearRegression)                0.99               0.99   
GenericBooster(Ridge)                           0.99               0.99   
GenericBooster(RidgeCV)                         0.99               0.99   
GenericBooster(TransformedTargetRegressor)      0.99               0.99   
GenericBooster(KernelRidge)                     0.97               0.96   
XGBClassifier                                   0.96               0.96   
GenericBooster(ExtraTreeRegressor)              0.94               0.94   
RandomForestClassifier                          0.92               0.93   
GenericBooster(RANSACRegressor)                 0.90               0.86   
GenericBooster(DecisionTreeRegressor)           0.87               0.88   
GenericBooster(KNeighborsRegressor)             0.87               0.89   
GenericBooster(ElasticNet)                      0.85               0.76   
GenericBooster(Lasso)                           0.82               0.71   
GenericBooster(LassoLars)                       0.82               0.71   
GenericBooster(DummyRegressor)                  0.68               0.50   

                                            ROC AUC  F1 Score  Time Taken  
Model                                                                      
GenericBooster(LinearRegression)               0.99      0.99        0.35  
GenericBooster(Ridge)                          0.99      0.99        0.27  
GenericBooster(RidgeCV)                        0.99      0.99        1.07  
GenericBooster(TransformedTargetRegressor)     0.99      0.99        0.40  
GenericBooster(KernelRidge)                    0.96      0.97        2.05  
XGBClassifier                                  0.96      0.96        0.91  
GenericBooster(ExtraTreeRegressor)             0.94      0.94        0.25  
RandomForestClassifier                         0.93      0.92        0.40  
GenericBooster(RANSACRegressor)                0.86      0.90       15.22  
GenericBooster(DecisionTreeRegressor)          0.88      0.87        0.98  
GenericBooster(KNeighborsRegressor)            0.89      0.87        0.49  
GenericBooster(ElasticNet)                     0.76      0.84        0.10  
GenericBooster(Lasso)                          0.71      0.79        0.09  
GenericBooster(LassoLars)                      0.71      0.79        0.10  
GenericBooster(DummyRegressor)                 0.50      0.56        0.01

2it [00:00,  8.29it/s]
100%|██████████| 30/30 [00:15<00:00,  1.92it/s]


Elapsed: 15.911818265914917 seconds



Accuracy  Balanced Accuracy  \
Model                                                                     
RandomForestClassifier                          1.00               1.00   
GenericBooster(ExtraTreeRegressor)              1.00               1.00   
GenericBooster(KernelRidge)                     1.00               1.00   
GenericBooster(LinearRegression)                1.00               1.00   
GenericBooster(Ridge)                           1.00               1.00   
GenericBooster(RidgeCV)                         1.00               1.00   
GenericBooster(TransformedTargetRegressor)      1.00               1.00   
XGBClassifier                                   0.97               0.96   
GenericBooster(Lars)                            0.94               0.94   
GenericBooster(DecisionTreeRegressor)           0.92               0.92   
GenericBooster(KNeighborsRegressor)             0.92               0.93   
GenericBooster(RANSACRegressor)                 0.81               0.81   
GenericBooster(ElasticNet)                      0.61               0.53   
GenericBooster(DummyRegressor)                  0.42               0.33   
GenericBooster(Lasso)                           0.42               0.33   
GenericBooster(LassoLars)                       0.42               0.33   

                                           ROC AUC  F1 Score  Time Taken  
Model                                                                     
RandomForestClassifier                        None      1.00        0.18  
GenericBooster(ExtraTreeRegressor)            None      1.00        0.16  
GenericBooster(KernelRidge)                   None      1.00        0.38  
GenericBooster(LinearRegression)              None      1.00        0.23  
GenericBooster(Ridge)                         None      1.00        0.17  
GenericBooster(RidgeCV)                       None      1.00        0.24  
GenericBooster(TransformedTargetRegressor)    None      1.00        0.26  
XGBClassifier                                 None      0.97        0.06  
GenericBooster(Lars)                          None      0.95        0.99  
GenericBooster(DecisionTreeRegressor)         None      0.92        0.23  
GenericBooster(KNeighborsRegressor)           None      0.92        0.21  
GenericBooster(RANSACRegressor)               None      0.80       12.63  
GenericBooster(ElasticNet)                    None      0.53        0.04  
GenericBooster(DummyRegressor)                None      0.25        0.01  
GenericBooster(Lasso)                         None      0.25        0.02  
GenericBooster(LassoLars)                     None      0.25        0.01

2it [00:00,  5.14it/s]
100%|██████████| 30/30 [00:15<00:00,  1.92it/s]


Elapsed: 16.0275661945343 seconds



Accuracy  Balanced Accuracy  \
Model                                                                     
GenericBooster(Ridge)                           1.00               1.00   
GenericBooster(RidgeCV)                         1.00               1.00   
RandomForestClassifier                          0.97               0.97   
XGBClassifier                                   0.97               0.97   
GenericBooster(DecisionTreeRegressor)           0.97               0.97   
GenericBooster(ExtraTreeRegressor)              0.97               0.97   
GenericBooster(LinearRegression)                0.97               0.97   
GenericBooster(TransformedTargetRegressor)      0.97               0.97   
GenericBooster(KNeighborsRegressor)             0.93               0.95   
GenericBooster(KernelRidge)                     0.87               0.83   
GenericBooster(RANSACRegressor)                 0.63               0.59   
GenericBooster(Lars)                            0.50               0.46   
GenericBooster(DummyRegressor)                  0.27               0.33   
GenericBooster(ElasticNet)                      0.27               0.33   
GenericBooster(Lasso)                           0.27               0.33   
GenericBooster(LassoLars)                       0.27               0.33   

                                           ROC AUC  F1 Score  Time Taken  
Model                                                                     
GenericBooster(Ridge)                         None      1.00        0.23  
GenericBooster(RidgeCV)                       None      1.00        0.25  
RandomForestClassifier                        None      0.97        0.26  
XGBClassifier                                 None      0.97        0.12  
GenericBooster(DecisionTreeRegressor)         None      0.97        0.27  
GenericBooster(ExtraTreeRegressor)            None      0.97        0.22  
GenericBooster(LinearRegression)              None      0.97        0.15  
GenericBooster(TransformedTargetRegressor)    None      0.97        0.37  
GenericBooster(KNeighborsRegressor)           None      0.93        1.52  
GenericBooster(KernelRidge)                   None      0.85        0.63  
GenericBooster(RANSACRegressor)               None      0.61       10.86  
GenericBooster(Lars)                          None      0.48        0.99  
GenericBooster(DummyRegressor)                None      0.11        0.01  
GenericBooster(ElasticNet)                    None      0.11        0.01  
GenericBooster(Lasso)                         None      0.11        0.01  
GenericBooster(LassoLars)                     None      0.11        0.01

In [ ]:
!pip install shap

In [ ]:
import shap

best_model = clf.get_best_model()

# load JS visualization code to notebook
shap.initjs()

# explain all the predictions in the test set
explainer = shap.KernelExplainer(best_model.predict_proba, X_train)
shap_values = explainer.shap_values(X_test)
# this is multiclass so we only visualize the contributions to first class (hence index 0)
shap.force_plot(explainer.expected_value[0], shap_values[..., 0], X_test)

  0%|          | 0/30 [00:00<?, ?it/s]

In [5]:
import os
import mlsauce as ms
from sklearn.datasets import load_diabetes
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split


data = load_diabetes()
X = data.data
y= data.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state = 123)

regr = ms.LazyBoostingRegressor(verbose=0, ignore_warnings=True,
                                custom_metric=None, preprocess=True)
models, predictioms = regr.fit(X_train, X_test, y_train, y_test)
model_dictionary = regr.provide_models(X_train, X_test, y_train, y_test)
display(models)

3it [00:00,  4.75it/s]
100%|██████████| 30/30 [00:58<00:00,  1.95s/it]


Adjusted R-Squared  R-Squared  \
Model                                                                       
GenericBooster(HuberRegressor)                            0.55       0.60   
GenericBooster(SGDRegressor)                              0.55       0.60   
GenericBooster(RidgeCV)                                   0.54       0.59   
GenericBooster(LinearSVR)                                 0.54       0.59   
GenericBooster(PassiveAggressiveRegressor)                0.54       0.59   
GenericBooster(Ridge)                                     0.54       0.59   
GenericBooster(TransformedTargetRegressor)                0.54       0.59   
GenericBooster(LinearRegression)                          0.54       0.59   
GenericBooster(KernelRidge)                               0.53       0.59   
GenericBooster(TweedieRegressor)                          0.53       0.59   
GenericBooster(LassoLars)                                 0.53       0.58   
GenericBooster(Lasso)                                     0.53       0.58   
GenericBooster(ElasticNet)                                0.53       0.58   
GenericBooster(SVR)                                       0.52       0.57   
GenericBooster(BayesianRidge)                             0.50       0.56   
GenericBooster(LassoLarsIC)                               0.49       0.55   
GradientBoostingRegressor                                 0.49       0.55   
GenericBooster(ElasticNetCV)                              0.49       0.55   
GenericBooster(LassoLarsCV)                               0.49       0.55   
GenericBooster(LassoCV)                                   0.49       0.55   
GenericBooster(LarsCV)                                    0.49       0.54   
GenericBooster(NuSVR)                                     0.46       0.53   
RandomForestRegressor                                     0.46       0.52   
GenericBooster(RANSACRegressor)                           0.44       0.50   
GenericBooster(ExtraTreeRegressor)                        0.41       0.47   
XGBRegressor                                              0.31       0.39   
GenericBooster(DecisionTreeRegressor)                     0.28       0.36   
GenericBooster(Lars)                                      0.19       0.28   
GenericBooster(DummyRegressor)                           -0.13      -0.00   
GenericBooster(QuantileRegressor)                        -0.15      -0.02   
GenericBooster(KNeighborsRegressor)                      -7.86      -6.85   

                                             RMSE  Time Taken  
Model                                                          
GenericBooster(HuberRegressor)              50.13        3.73  
GenericBooster(SGDRegressor)                50.40        0.36  
GenericBooster(RidgeCV)                     50.53        0.40  
GenericBooster(LinearSVR)                   50.54        0.18  
GenericBooster(PassiveAggressiveRegressor)  50.63        0.30  
GenericBooster(Ridge)                       50.70        0.31  
GenericBooster(TransformedTargetRegressor)  50.75        0.46  
GenericBooster(LinearRegression)            50.75        0.39  
GenericBooster(KernelRidge)                 50.99        3.09  
GenericBooster(TweedieRegressor)            51.10        0.66  
GenericBooster(LassoLars)                   51.17        0.44  
GenericBooster(Lasso)                       51.17        0.20  
GenericBooster(ElasticNet)                  51.24        0.31  
GenericBooster(SVR)                         51.97        3.54  
GenericBooster(BayesianRidge)               52.93        0.97  
GenericBooster(LassoLarsIC)                 53.20        0.39  
GradientBoostingRegressor                   53.23        0.14  
GenericBooster(ElasticNetCV)                53.43        3.73  
GenericBooster(LassoLarsCV)                 53.44        1.23  
GenericBooster(LassoCV)                     53.45        4.01  
GenericBooster(LarsCV)                      53.54        0.90  
GenericBooster(NuSVR)         

In [10]:
data = fetch_california_housing()
n_points = 1000
idx_inputs = range(n_points)
X = data.data[idx_inputs,:]
y= data.target[idx_inputs]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state = 123)

regr = ms.LazyBoostingRegressor(verbose=0, ignore_warnings=True,
                                custom_metric=None, preprocess=True)
models, predictioms = regr.fit(X_train, X_test, y_train, y_test)
model_dictionary = regr.provide_models(X_train, X_test, y_train, y_test)
display(models)

3it [00:03,  1.01s/it]
100%|██████████| 30/30 [02:32<00:00,  5.10s/it]


Adjusted R-Squared  R-Squared  \
Model                                                                       
GenericBooster(ExtraTreeRegressor)                        0.82       0.83   
RandomForestRegressor                                     0.82       0.83   
GradientBoostingRegressor                                 0.79       0.79   
GenericBooster(NuSVR)                                     0.79       0.79   
GenericBooster(SVR)                                       0.78       0.79   
XGBRegressor                                              0.78       0.79   
GenericBooster(HuberRegressor)                            0.77       0.78   
GenericBooster(LinearSVR)                                 0.77       0.77   
GenericBooster(TransformedTargetRegressor)                0.75       0.76   
GenericBooster(LinearRegression)                          0.75       0.76   
GenericBooster(Ridge)                                     0.75       0.76   
GenericBooster(RANSACRegressor)                           0.75       0.76   
GenericBooster(RidgeCV)                                   0.75       0.76   
GenericBooster(PassiveAggressiveRegressor)                0.74       0.75   
GenericBooster(SGDRegressor)                              0.73       0.74   
GenericBooster(DecisionTreeRegressor)                     0.73       0.74   
GenericBooster(KernelRidge)                               0.71       0.72   
GenericBooster(LassoLarsIC)                               0.71       0.72   
GenericBooster(BayesianRidge)                             0.71       0.72   
GenericBooster(LassoLarsCV)                               0.70       0.71   
GenericBooster(LassoCV)                                   0.70       0.71   
GenericBooster(ElasticNetCV)                              0.70       0.71   
GenericBooster(TweedieRegressor)                          0.69       0.71   
GenericBooster(LarsCV)                                    0.69       0.70   
GenericBooster(Lars)                                      0.42       0.44   
GenericBooster(ElasticNet)                                0.25       0.28   
GenericBooster(QuantileRegressor)                        -0.04      -0.00   
GenericBooster(Lasso)                                    -0.08      -0.04   
GenericBooster(DummyRegressor)                           -0.08      -0.04   
GenericBooster(LassoLars)                                -0.08      -0.04   
GenericBooster(KNeighborsRegressor)                      -0.46      -0.40   

                                            RMSE  Time Taken  
Model                                                         
GenericBooster(ExtraTreeRegressor)          0.34        0.93  
RandomForestRegressor                       0.34        1.27  
GradientBoostingRegressor                   0.37        0.28  
GenericBooster(NuSVR)                       0.37       18.97  
GenericBooster(SVR)                         0.38       15.78  
XGBRegressor                                0.38        1.48  
GenericBooster(HuberRegressor)              0.39        5.49  
GenericBooster(LinearSVR)                   0.39        7.15  
GenericBooster(TransformedTargetRegressor)  0.40        3.12  
GenericBooster(LinearRegression)            0.40        1.94  
GenericBooster(Ridge)                       0.40        0.48  
GenericBooster(RANSACRegressor)             0.41       32.76  
GenericBooster(RidgeCV)                     0.41        2.54  
GenericBooster(PassiveAggressiveRegressor)  0.41        0.55  
GenericBooster(SGDRegressor)                0.42        0.66  
GenericBooster(DecisionTreeRegressor)       0.42        2.48  
GenericBooster(KernelRidge)                 0.43       13.27  
GenericBooster(LassoLarsIC)                 0.44        1.33  
GenericBooster(BayesianRidge)               0.44        2.82  
GenericBooster(LassoLarsCV)                 0.44        2.51  
GenericBooster(LassoCV)                     0.44        9.69  
GenericBooster(ElasticNetCV)                0.44     